# Clustering Models

First of all, we are going to import the necessary libraries.

In [2]:
import pandas as pd
from cuml import NearestNeighbors
import numpy as np
import optuna
import cuml
from cuml.cluster import DBSCAN as cuDBSCAN
from dask.array import asarray
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler
from cuml.metrics.cluster.silhouette_score import cython_silhouette_score as cu_silhouette_score
from sklearn.cluster import Birch
from sklearn.decomposition import PCA
import joblib
import optuna.visualization as vis
import cupy as cp
import numpy as np
from sklearn.cluster import Birch
from sklearn.decomposition import PCA

First of all, we are going to create a class to compute all the metrics. This class will be used to evaluate the performance of the models using the K Fold method.


In [19]:
import os

class ClusteringMetrics:
    def __init__(self, X, model):
        # self.X = pd.DataFrame(X.get()).reset_index(drop=True)

        if hasattr(X, 'get'):
            self.X = pd.DataFrame(X.get()).reset_index(drop=True)
        else:
            self.X = pd.DataFrame(X).reset_index(drop=True)
        self.model = model

    def train(self):
        self.model.fit(self.X)

    def compute_internalEvaluation(self):
        if hasattr(self.model, 'predict'):
            # Use predict method if available (e.g., KMeans, GaussianMixture)
            y_pred = self.model.predict(self.X)
        elif hasattr(self.model, 'labels_'):
            # Use labels_ for models like DBSCAN after fitting
            y_pred = self.model.labels_
        else:
            # Raise an error if the model doesn't have any compatible method
            raise AttributeError("The model does not have a valid method to predict cluster labels.")

        # print("Starting to compute metrics")
        self.shiloette_score = cu_silhouette_score(cp.asarray(self.X), cp.asarray(y_pred))
        # print("Shiloette score computed")
        self.calinski_harabasz_score = calinski_harabasz_score(self.X, y_pred)
        # print("Calinski Harabasz score computed")
        self.davies_bouldin_score = davies_bouldin_score(self.X, y_pred)
        # print("Davies Bouldin score computed")

    def print_metrics(self):
        print(f"Silhouette Score: {self.shiloette_score}")
        print(f"Calinski Harabasz Score: {self.calinski_harabasz_score}")
        print(f"Davies Bouldin Score: {self.davies_bouldin_score}")

    def save_model(self, path):
        directory = os.path.dirname(path)
        if not os.path.exists(directory):
            os.makedirs(directory)

        joblib.dump(self.model, path)



Loading the data

In [5]:
df = pd.read_parquet('../data/processed/selected_features_df.parquet')

We are going to scale the data using the StandardScaler.

In [6]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

Let's calculate the Hopkins statistic to check if the data is suitable for clustering.

In [5]:
def hopkins(X):
    d = X.shape[1]
    n = len(X)
    m = int(0.1 * n)
    nbrs = NearestNeighbors(n_neighbors=1).fit(X.values)

    rand_X = np.random.rand(n, d)
    ujd = []
    wjd = []

    for j in range(m):
        u_dist, _ = nbrs.kneighbors(rand_X[j].reshape(1, -1), 2, return_distance=True)
        ujd.append(u_dist[0][1])
        w_dist, _ = nbrs.kneighbors(X.iloc[j].values.reshape(1, -1), 2, return_distance=True)
        wjd.append(w_dist[0][1])

    H = sum(ujd) / (sum(ujd) + sum(wjd))
    if np.isnan(H):
        print(ujd, wjd)
        H = 0

    return H

The Hopkins statistic ranges from 0 to 1. The closer to 1, the more suitable the data is for clustering. Let's calculate the Hopkins statistic for the data.

In [14]:
hopkins(df)

0.9645847043701986

The Hopkins statistic is close to 1, so the data is suitable for clustering.

# DBSCAN

We are going to use the DBSCAN algorithm to cluster the data. We are going to use the Optuna library to find the best hyperparameters.

In [6]:
import cupy as cp

def objective(trial):
    eps = trial.suggest_float('eps', 0.1, 1.0)
    min_samples = trial.suggest_int('min_samples', 5, 10)
    metric = trial.suggest_categorical('metric', ['euclidean', 'cosine'])

    model = cuDBSCAN(eps=eps, min_samples=min_samples, metric=metric)
    model.fit(df_scaled)
    y_pred = model.labels_

    # Verificar si hay al menos 2 clusters
    if len(cp.unique(y_pred)) > 1:  # Usamos cupy para manejar el arreglo en GPU
        return silhouette_score(df_scaled, y_pred)  # Devuelve el Silhouette score si hay más de 1 cluster
    else:
        return -1  # Devuelve un valor negativo si solo se encuentra un único cluster
    study = optuna.create_study(direction='maximize', study_name='dbscan')
    study.optimize(objective, n_trials=5)
    print(f"Best parameters: {study.best_params}")

[I 2024-12-06 14:29:53,352] A new study created in memory with name: dbscan
[I 2024-12-06 14:46:08,489] Trial 0 finished with value: 0.06344437831203903 and parameters: {'eps': 0.45560417786879226, 'min_samples': 9, 'metric': 'cosine'}. Best is trial 0 with value: 0.06344437831203903.
[I 2024-12-06 15:01:01,110] Trial 1 finished with value: -0.08649164881054881 and parameters: {'eps': 0.2812818251401415, 'min_samples': 9, 'metric': 'cosine'}. Best is trial 0 with value: 0.06344437831203903.
[I 2024-12-06 15:05:37,931] Trial 2 finished with value: -1.0 and parameters: {'eps': 0.15472718863535367, 'min_samples': 10, 'metric': 'euclidean'}. Best is trial 0 with value: 0.06344437831203903.
[I 2024-12-06 15:20:50,951] Trial 3 finished with value: 0.12279910760197582 and parameters: {'eps': 0.529945149343017, 'min_samples': 10, 'metric': 'cosine'}. Best is trial 3 with value: 0.12279910760197582.
[I 2024-12-06 15:36:01,502] Trial 4 finished with value: 0.15272329032663132 and parameters: {'e

Best parameters: {'eps': 0.9095320672300284, 'min_samples': 8, 'metric': 'cosine'}


We can see that the bigger the eps, the better the silhouette score. Also, the cosine metric is better than the euclidean metric for this problem. Taking this into account we are going to create another Optuna study to find the best hyperparameters.

In [11]:
import cupy as cp
import gc

df_scaled = cp.asarray(df_scaled)

def objective(trial):
    gc.collect()
    cp._default_memory_pool.free_all_blocks()

    eps = trial.suggest_float('eps', 1, 1.5)
    min_samples = trial.suggest_int('min_samples', 5, 10)

    model = cuDBSCAN(eps=eps, min_samples=min_samples, metric='cosine')
    model.fit(df_scaled)
    y_pred = model.labels_

    # Verificar si hay al menos 2 clusters
    if len(cp.unique(y_pred)) > 1:  # Usamos cupy para manejar el arreglo en GPU
        return cu_silhouette_score(df_scaled, y_pred)  # Devuelve el Silhouette score si hay más de 1 cluster
    else:
        return -1  # Devuelve un valor negativo si solo se encuentra un único cluster
study = optuna.create_study(direction='maximize', study_name='dbscan')
study.optimize(objective, n_trials=5)
print(f"Best parameters: {study.best_params}")

[I 2024-12-07 10:09:25,235] A new study created in memory with name: dbscan
[I 2024-12-07 10:18:00,067] Trial 0 finished with value: 0.35992020990680346 and parameters: {'eps': 1.468031449624661, 'min_samples': 9}. Best is trial 0 with value: 0.35992020990680346.
[I 2024-12-07 10:26:35,946] Trial 1 finished with value: 0.37549860352774733 and parameters: {'eps': 1.2027419903680259, 'min_samples': 8}. Best is trial 1 with value: 0.37549860352774733.
[I 2024-12-07 10:35:13,500] Trial 2 finished with value: 0.38383985601186843 and parameters: {'eps': 1.2125524450750875, 'min_samples': 7}. Best is trial 2 with value: 0.38383985601186843.
[I 2024-12-07 10:43:59,112] Trial 3 finished with value: 0.35992020990680346 and parameters: {'eps': 1.4344474832899103, 'min_samples': 6}. Best is trial 2 with value: 0.38383985601186843.
[I 2024-12-07 10:52:46,669] Trial 4 finished with value: 0.1540651024846537 and parameters: {'eps': 1.001327955535295, 'min_samples': 6}. Best is trial 2 with value: 0.3

Best parameters: {'eps': 1.2125524450750875, 'min_samples': 7}


In [20]:
model = cuDBSCAN(eps=1.2125524450750875, min_samples=7, metric='cosine')
metrics = ClusteringMetrics(df_scaled, model)
metrics.train()
metrics.save_model('../models/dbscan.pkl')
metrics.compute_internalEvaluation()
metrics.print_metrics()

Silhouette Score: 0.38383985601186843
Calinski Harabasz Score: 6.71393677535788
Davies Bouldin Score: 1.5755187609143613


# BIRCH

In [14]:
pca = PCA(n_components=4)  # Adjust this number to reduce dimensionality
df_scaled_reduced = pca.fit_transform(df_scaled)

In [8]:
def objective(trial):
    # Sugerir valores para los hiperparámetros de BIRCH
    threshold = trial.suggest_float('threshold', 0.01, 1.0)  # Umbral para la construcción del árbol
    # n_clusters = trial.suggest_int('n_clusters', 2, 5)  # Número de clusters
    branching_factor = trial.suggest_int('branching_factor', 10, 100)  # Número de hijos por nodo

    model = Birch(threshold=threshold, n_clusters=3, branching_factor=branching_factor)

    # Entrenar el modelo
    model.fit(df_scaled_reduced)
    labels = model.labels_

    # Calcular el Silhouette Score para evaluar la calidad del clustering
    if len(np.unique(labels)) > 1:
        score = silhouette_score(df_scaled_reduced, labels)
        return score
    else:
        return -1

study = optuna.create_study(direction='maximize', study_name='birch_tuning')
study.optimize(objective, n_trials=5)
print(f"Best parameters: {study.best_params}")

[I 2024-12-07 16:39:45,614] A new study created in memory with name: birch_tuning
[I 2024-12-07 16:49:00,660] Trial 0 finished with value: 0.34914341637482804 and parameters: {'threshold': 0.5815544590284647, 'branching_factor': 65}. Best is trial 0 with value: 0.34914341637482804.
[I 2024-12-07 16:58:12,682] Trial 1 finished with value: 0.3591458222679078 and parameters: {'threshold': 0.7481263824689723, 'branching_factor': 98}. Best is trial 1 with value: 0.3591458222679078.
[I 2024-12-07 17:07:27,574] Trial 2 finished with value: 0.40124587744456613 and parameters: {'threshold': 0.9963391215347926, 'branching_factor': 25}. Best is trial 2 with value: 0.40124587744456613.
[I 2024-12-07 17:16:40,971] Trial 3 finished with value: 0.3438450715954416 and parameters: {'threshold': 0.7656417232067172, 'branching_factor': 14}. Best is trial 2 with value: 0.40124587744456613.
[I 2024-12-07 17:25:59,779] Trial 4 finished with value: 0.3469181031941854 and parameters: {'threshold': 0.392678476

In [9]:
from sklearn.cluster import Birch
from sklearn.decomposition import PCA

pca = PCA(n_components=4)  # Adjust this number to reduce dimensionality
df_scaled_reduced = pca.fit_transform(df_scaled)

def objective(trial):
    # Sugerir valores para los hiperparámetros de BIRCH
    threshold = trial.suggest_float('threshold', 0.01, 1.0)  # Umbral para la construcción del árbol
    # n_clusters = trial.suggest_int('n_clusters', 2, 5)  # Número de clusters
    branching_factor = trial.suggest_int('branching_factor', 10, 100)  # Número de hijos por nodo

    model = Birch(threshold=threshold, n_clusters=3, branching_factor=branching_factor)

    # Entrenar el modelo
    model.fit(df_scaled_reduced)
    labels = model.labels_

    # Calcular el Silhouette Score para evaluar la calidad del clustering
    if len(np.unique(labels)) > 1:
        score = silhouette_score(df_scaled_reduced, labels)
        return score
    else:
        return -1

study = optuna.create_study(direction='maximize', study_name='birch_tuning')
study.optimize(objective, n_trials=5)

[I 2024-12-07 17:26:41,618] A new study created in memory with name: birch_tuning
[I 2024-12-07 17:35:48,675] Trial 0 finished with value: 0.34974572749924693 and parameters: {'threshold': 0.9003977028532181, 'branching_factor': 99}. Best is trial 0 with value: 0.34974572749924693.
[I 2024-12-07 17:45:02,581] Trial 1 finished with value: 0.3417716772528243 and parameters: {'threshold': 0.5573442455641573, 'branching_factor': 51}. Best is trial 0 with value: 0.34974572749924693.
[I 2024-12-07 17:54:14,660] Trial 2 finished with value: 0.33049820651172246 and parameters: {'threshold': 0.9302214872562525, 'branching_factor': 93}. Best is trial 0 with value: 0.34974572749924693.
[I 2024-12-07 18:03:31,409] Trial 3 finished with value: 0.4034668986672024 and parameters: {'threshold': 0.7238227929156842, 'branching_factor': 100}. Best is trial 3 with value: 0.4034668986672024.
[I 2024-12-07 18:12:46,777] Trial 4 finished with value: 0.3954343902201493 and parameters: {'threshold': 0.82445205

In [11]:
pca = PCA(n_components=4)  # Adjust this number to reduce dimensionality
df_scaled_reduced = pca.fit_transform(df_scaled)

def objective(trial):
    # Sugerir valores para los hiperparámetros de BIRCH
    threshold = trial.suggest_float('threshold', 1.83, 3.0)  # Umbral para la construcción del árbol
    # n_clusters = trial.suggest_int('n_clusters', 2, 5)  # Número de clusters
    branching_factor = trial.suggest_int('branching_factor', 10, 100)  # Número de hijos por nodo

    model = Birch(threshold=threshold, n_clusters=3, branching_factor=branching_factor)

    # Entrenar el modelo
    model.fit(df_scaled_reduced)
    labels = model.labels_

    # Calcular el Silhouette Score para evaluar la calidad del clustering
    if len(np.unique(labels)) > 1:
        score = silhouette_score(df_scaled_reduced, labels)
        return score
    else:
        return -1

study = optuna.create_study(direction='maximize', study_name='birch_tuning')
study.optimize(objective, n_trials=5)

[I 2024-12-07 19:01:59,643] A new study created in memory with name: birch_tuning
[I 2024-12-07 19:11:21,279] Trial 0 finished with value: 0.18575780860921098 and parameters: {'threshold': 2.60466881925551, 'branching_factor': 46}. Best is trial 0 with value: 0.18575780860921098.
[I 2024-12-07 19:20:31,179] Trial 1 finished with value: 0.3564872887967888 and parameters: {'threshold': 2.1540956836508287, 'branching_factor': 83}. Best is trial 1 with value: 0.3564872887967888.
[I 2024-12-07 19:29:52,822] Trial 2 finished with value: 0.41516733727246385 and parameters: {'threshold': 2.228654160023864, 'branching_factor': 46}. Best is trial 2 with value: 0.41516733727246385.
[I 2024-12-07 19:39:07,647] Trial 3 finished with value: 0.3983764555116041 and parameters: {'threshold': 2.7756256704572326, 'branching_factor': 61}. Best is trial 2 with value: 0.41516733727246385.
[I 2024-12-07 19:48:14,067] Trial 4 finished with value: 0.338279702613758 and parameters: {'threshold': 2.7736347980784

Computing the metrics for the best parameters.

In [15]:
model = Birch(threshold=2.228654160023864, n_clusters=3, branching_factor=46)
model.fit(df_scaled_reduced)
#Save model
# joblib.dump(model, '../models/birch.pkl')

metrics = ClusteringMetrics(df_scaled_reduced, model)
metrics.train()
metrics.compute_internalEvaluation()
metrics.save_model('../models/birch.pkl')
metrics.print_metrics()

Silhouette Score: 0.41516733727246374
Calinski Harabasz Score: 69263.9973281745
Davies Bouldin Score: 0.9181020942854774


# Gaussian Mixture

We are going to use the Gaussian Mixture algorithm to cluster the data. We are going to use the Optuna library to find the best hyperparameters.

In [9]:
from sklearn.mixture import GaussianMixture

def objective(trial):
    n_components = trial.suggest_int('n_components', 2, 5)
    covariance_type = trial.suggest_categorical('covariance_type', ['full', 'tied', 'diag', 'spherical'])
    tol = trial.suggest_float('tol', 1e-5, 1e-1)
    reg_covar = trial.suggest_float('reg_covar', 1e-6, 1e-2)


    model = GaussianMixture(n_components=n_components, covariance_type=covariance_type, tol=tol, reg_covar=reg_covar, random_state=42)
    model.fit(df_scaled)
    y_pred = model.predict(df_scaled)

    return silhouette_score(df_scaled, y_pred)

study = optuna.create_study(direction='maximize', study_name='gmm')
study.optimize(objective, n_trials=5)

[I 2024-12-10 21:16:58,431] A new study created in memory with name: gmm
[I 2024-12-10 21:26:26,832] Trial 0 finished with value: 0.068944020380107 and parameters: {'n_components': 2, 'covariance_type': 'tied', 'tol': 0.02792127136333412, 'reg_covar': 0.007863629516834955}. Best is trial 0 with value: 0.068944020380107.
[I 2024-12-10 21:35:01,073] Trial 1 finished with value: 0.06353784088589444 and parameters: {'n_components': 5, 'covariance_type': 'diag', 'tol': 0.02418755280161164, 'reg_covar': 0.0006082345442615693}. Best is trial 0 with value: 0.068944020380107.
[I 2024-12-10 21:43:37,780] Trial 2 finished with value: 0.03497509670255373 and parameters: {'n_components': 3, 'covariance_type': 'spherical', 'tol': 0.0444942314395488, 'reg_covar': 0.009547804964557524}. Best is trial 0 with value: 0.068944020380107.
[I 2024-12-10 21:53:19,753] Trial 3 finished with value: 0.06340238645125286 and parameters: {'n_components': 2, 'covariance_type': 'spherical', 'tol': 0.02765508096564704

Computing the metrics for the best parameters.

In [37]:
model = GaussianMixture(**study.best_params, random_state=42)

metrics = ClusteringMetrics(df_scaled, model)
metrics.train()
metrics.compute_internalEvaluation()
metrics.save_model('../models/gmm.pkl')
metrics.print_metrics()

Silhouette Score: 0.06894402038010695
Calinski Harabasz Score: 19626.39283621834
Davies Bouldin Score: 2.5051416177208856


In [13]:
model = joblib.load('../models/gmm.pkl')

metrics = ClusteringMetrics(df_scaled, model)
metrics.compute_internalEvaluation()
metrics.print_metrics()

Silhouette Score: 0.06894402038010695
Calinski Harabasz Score: 19626.39283621834
Davies Bouldin Score: 2.5051416177208856


Plot the study results.

In [ ]:
vis.plot_optimization_history(study)


In [ ]:
vis.plot_parallel_coordinate(study)

In [ ]:
vis.plot_slice(study)

In [ ]:
vis.plot_param_importances(study)

In [ ]:
vis.plot_contour(study)